### README
- From the user favourites page in [つくおき](https://cookien.com/favorite/)
- Extract all recipes found inside
  - Title
  - Storage period
  - Ingredients
  - Instructions

---
### Init selenium stuff

In [1]:
import os.path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Init chromedriver
chrome_options = Options()
chrome_options.add_argument("--headless") # turn off GUI
chrome_options.add_argument("--no-sandbox")

homedir = os.path.expanduser("~")
webdriver_service = Service(f"{homedir}/chromedriver/stable/chromedriver")

# Init browser
browser = webdriver.Chrome(service=webdriver_service, options=chrome_options)

---

### Open favorites page

In [3]:
browser.get("https://cookien.com/favorite/")

elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@id='wpmem_login']")))
print(browser.title)

print(elems[0].find_element(By.XPATH, "//div[@id='wpmem_login']").text)

| つくおき | 作り置き・簡単レシピサイト
ログイン
ユーザID または メールアドレス
パスワード
 ログイン状態を保存する  
パスワードをお忘れですか？ パスワードリセット
はじめての方はこちら 新規ユーザー登録


---
### Handle login

In [10]:
import getpass
username = input("Username:")
password = getpass.getpass("Password:")

In [11]:
# Login
login_username = elems[0].find_element(By.XPATH, "//input[@id='log']")
login_password = elems[0].find_element(By.XPATH, "//input[@id='pwd']")
login_submit = elems[0].find_element(By.XPATH, "//input[@type='submit'][@class='buttons']")

login_username.send_keys(username)
login_password.send_keys(password)

login_submit.click()

# Post-login / wait for page to load
print(browser.title)
elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='site-info clear']")))
print(elems[0].find_element(By.XPATH, "//div[@class='site-info clear']").text)

お気に入り一覧 | つくおき | 作り置き・簡単レシピサイト
© 2013-2023 つくおき All rights reserved.
当サイトの情報及び、使用している画像、テキスト等の無断転載を禁じます。


--- 

### Extract recipe data for each item found in favourites page

In [12]:
elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='articleList']")))
for elem in elems[0].find_elements(By.XPATH, ".//a[@class='content_list_link']"):
    elem_text = elem.get_attribute("title")
    elem_url = elem.get_attribute("href")
    print(elem_text, elem_url)

    # Open url in new tab
    original_window = browser.current_window_handle
    assert len(browser.window_handles) == 1
    browser.switch_to.new_window('tab')
    browser.get(elem_url)  

    # Get info
    sub_elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@id='content']")))
    print("#", sub_elems[0].find_element(By.XPATH, ".//div[@id='clock_reizouko']").text)
    print("#", sub_elems[0].find_element(By.XPATH, ".//div[@id='r_contents']/h2").text.replace('\n', ', '))
    for ingredients in sub_elems[0].find_elements(By.XPATH, ".//div[@id='r_contents']/p"):
        print("\t", ingredients.text.replace('\n', ' '))

    print("#", sub_elems[0].find_element(By.XPATH, ".//div[@id='ins_contents']/h2").text.replace('\n', ', '))
    for instructions in sub_elems[0].find_elements(By.XPATH, ".//p[@class='ins_des']"):
        print("\t", instructions.text.replace('\n', ' '))

    # Close tab and go back to main window
    browser.close()
    browser.switch_to.window(original_window)
    print(50*"=")

小松菜のわさび風味おひたし https://cookien.com/recipe/10347/
# 10分日持ち：冷蔵4日
# 材料（保存容器中1個分）, 食べきりの場合 3～4人分
	 小松菜 1袋（5～7株くらい）
	 ◎白だし 大1/2
	 ◎醤油 小1
	 ◎わさびチューブ 3～4cm（メモ1）
# 作り方
	 鍋に水（1L）、塩（小1）を入れ、強めの中火で沸かします。小松菜は根部分の軸を切り落とし、水洗いします。
	 湯が沸騰したら、茎を入れて30秒、葉も入れてさらに30秒ゆでます。ゆでたら、ざるにあげてあら熱を取ります。
	 全体をぎゅっとしぼって水気を切ります。根元を切り落とし、3～4等分に切り分け、再度しっかりと水気を切ります。（メモ2）
	 ◎はボウルでよく混ぜ合わせます。小松菜を入れ、菜ばしでほぐしながらよく和えて、完成。（メモ3）
小松菜とちくわとひじきの和風サラダ https://cookien.com/recipe/36524/
# 15分日持ち：冷蔵4日
# 材料（保存容器中1個分）, 食べきりの場合 3～4人分
	 小松菜 1袋（5株くらい）
	 ちくわ 2本（約60g）
	 乾燥芽ひじき 2g
	 塩 少々
	 ◎めんつゆ（3倍濃厚） 大1
	 ◎ごま油 大1
	 ◎おかか 小分け1袋（2.5g）
# 作り方
	 小松菜は水洗いして水気を切り、根元を切り落とし、1～2cm幅に切ります。ちくわは薄切りにします。ひじきは沸騰させた湯に10分ほどさらして戻します。（メモ1）
	 耐熱皿に小松菜を広げて入れます。塩をふり、ふわりとラップをし、500wの電子レンジで4分加熱します。加熱したらざるにあげ、手でしぼってしっかりと水気を切ります。（メモ2）
	 ボウルに小松菜、ちくわ、水気を切ったひじき、◎を入れ、よく和えて、完成。（メモ3）
小松菜のナムル https://cookien.com/recipe/2335/
# 10分日持ち：冷蔵4日
# 材料（保存容器中1個分）, 食べきりの場合 3～4人分
	 小松菜 1袋（5～7株くらい）
	 ◎いりごま 大1
	 ◎ごま油 大1
	 ◎醤油 小1
	 ◎中華スープの素（メモ1） 2.5g
	 ◎にんにくチューブ 2cm
# 作り方
	 鍋に水（1L）、塩（小1）を入れ、強めの中火で沸かします。

In [13]:
browser.quit()